In [1]:
import os
# os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
import os

import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import psycopg2
import pandas as pd
from sqlalchemy import create_engine , text
from typing import Optional
from sklearn.model_selection import train_test_split
import boto3
import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from azure.storage.blob import BlockBlobService
from io import StringIO


In [2]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = ''
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['MLFLOW_BASE_URL'] = ''

# connect to remote server
mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])

# Launch the experiment on mlflow
experiment_name = "cust-lifetime"
mlflow.set_experiment(experiment_name)

exp_details = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp_details.experiment_id

client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [3]:
MODEL_NAME = "customer-lifetime-value"
#STORAGEACCOUNTNAME= ""
#STORAGEACCOUNTKEY= ""    
#CONTAINERNAME= ""

POSTGRES_ADDRESS = '' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = ''
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = '' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = ''
PROJECT_NAME = 'katonic'
PROJECT_SCHEMA = 'usecase'
TABLE_DATA = 'customer_lifetime_value_pre'
TABLE_MODEL = 'customer_lifetime_value_model'
TABLE_PRED = 'customer_lifetime_pred'

In [4]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD


def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()

def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()

In [5]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
VAL = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,"table_val").iloc[::-1].reset_index(drop=True)
VAL.head()

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "52.187.120.22" and accepting
	TCP/IP connections on port 5432?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
VAL.columns

In [ ]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
model_df = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_MODEL).iloc[::-1].reset_index(drop=True)
model_df.head()

In [ ]:
exp_id = model_df['experiment_id'][0]
model_name = model_df['model'][0]
run_id = model_df['run_id'][0]

In [ ]:
print('experiment_id :',exp_id)
print('=='*50)
print('model name :',model_name)
print('=='*50)
print('run_id :',run_id)

In [ ]:
#Etop_features = model_df['top_features'][0].split('{')[1].split('}')[0].split(',')
#print('Selected Features :',top_features)


In [ ]:
model_df.loc[0:,'location'][0]

In [ ]:
model = mlflow.sklearn.load_model(model_df.loc[0:,'location'][0])

In [ ]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
data = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_DATA)

data.head() 


In [ ]:
data.columns

In [ ]:
#top_features = model_df['top_features'].unique#[0].split('{')[1].split('}')[0].split(',')[1].split('"')
#print('Selected Features :',top_features)

In [ ]:
customer_id = VAL.pop('customer')
top_features=["income","monthly premium auto","total claim amount"]
inference_data= VAL[top_features]
prediction = model.predict(inference_data)
inference_data

In [ ]:
prediction

In [ ]:
#X = data['monthly premium auto'].values.reshape(-1,1)
#y = data['customer lifetime value'].values.reshape(-1,1)

In [ ]:
fg= pd.DataFrame(prediction)
fg
dd=VAL['customer lifetime value']
cust= VAL['customer']
main= pd.concat([cust,dd,fg],axis='columns')

In [ ]:
fg

In [ ]:
dd

In [ ]:
main
main.rename(columns={0: 'predicted value', 'customer lifetime value': 'actual value'})

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#model = LinearRegression()  
#model.fit(X_train, y_train)

In [ ]:
#print(model.intercept_)
#For retrieving the slope:
#print(model.coef_)

In [ ]:
#customer_id = data['customer lifetime value']

#inference_data = data['monthly premium auto']



In [ ]:
#y_pred = model.predict(X_test)

In [ ]:
#df_ = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
#df_

In [ ]:
#customer_id = data['customer lifetime value']

#inference_data = data['monthly premium auto']

In [ ]:
#prediction = model.predict(inference_data)
#prediction_probability = model.predict_proba(inference_data)[:,1]

In [ ]:
predicted_data = pd.DataFrame()
predicted_data['Experiment_ID'] = [exp_id]*len(customer_id)
predicted_data['Run_ID'] = [run_id]*len(customer_id)
predicted_data['Model_Name'] = [model_name]*len(customer_id)
predicted_data['Customer_ID'] = customer_id
predicted_data['predicted'] = prediction
#predicted_data['actual'] = df_['Actual']
#
##predicted_data['predicted-val']=prediction_probability 
predicted_data['Prediction_time'] =pd.to_datetime(datetime.now())
predicted_data.head()

In [ ]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,predicted_data,TABLE_PRED)